# Qasm - Imports

This notebook elaborates the [OpenQASM](https://github.com/Qiskit/openqasm) import and export functions. It can also serve as a short introduction to the QASM format.

In [1]:
from qutip.qip.qasm import read_qasm

The process for is quite simple and only requires the user to store the .qasm file in an appropriate location and maintain the absolute path of the file. This will make it simple for us to read the file. 
Let's start off with a simple qasm file.

In [2]:
path = "qasm_files/swap.qasm"
qasm_file = open(path, "r")
print(qasm_file.read())

// SWAP gate impemented in terms of cx's

OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];

cx q[1], q[0];
cx q[0], q[1];
cx q[1], q[0];



This QASM file imitates the iSWAP gate native to QuTiP in the QASM format. To read it, we use the `read_qasm` function with the arguments being the file path, the `mode` which defaults to qiskit and the `version` which defaults to "2.0".  

qc = read_qasm(path, mode="qiskit", version="2.0")

We can now check that the circuit indeed implements the swap gate by checking the unitary matrix corresponding
to the circuit. This can be done by using the `gate_sequence_product` function and the `propagators` function of the 
`QubitCircuit` class. 

In [3]:
from qutip.qip.operations.gates import gate_sequence_product
from qutip import tensor, basis

qc = read_qasm(path, mode="qiskit", version="2.0")
gate_sequence_product(qc.propagators()) 

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

The `mode` refers to the internal way in which QuTiP processes the QASM files. 
With qiskit mode, QASM skips the include command for the file qelib1.inc and converts all custom gates specified in them directly into QuTiP gates. It is recommended to use the QASM import function in this mode. 

The `version` refers to the version of the OpenQASM standard being processed. The documentation for the same can be found in the [OpenQASM](https://github.com/Qiskit/openqasm) repository. Currently, only OpenQASM 2.0 is supported which is the most popular QASM standard. 

### Custom Gates

QASM also offers the option to define custom gates in terms of already defined gates using the "gate" keyword. In qiskit mode, our QASM interpreter can be assumed to already allow for all the gates defined in the file [qelib1.inc](https://github.com/Qiskit/openqasm/blob/master/examples/generic/qelib1.inc) provided by the OpenQASM repository.

In the file `swap_custom.qasm`, we define the `swap` gate in terms of the pre-defined `cx` gates.

In [4]:
path = "qasm_files/swap_custom.qasm"
qasm_file = open(path, "r")
print(qasm_file.read())

// SWAP gate defined as a custom gate.

OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg c[2];

gate swap a, b{
cx b, a;
cx a, b;
cx b, a;
}

swap q[0], q[1]

measure q -> c



Furthermore, the circuit also measures the two qubits q[0] and q[1] and stores the results in the classical registers c[0] and c[1]

In [5]:
qc = read_qasm(path)

We can now run the circuit to confirm that the circuit is correctly loaded and performs the correct operations. To do this, we can use the `QubitCircuit.run` function with the appropriate input state. In our case, we can take the state `|01⟩`. 

In [6]:
from qutip import tensor, basis

qc.run(tensor(basis(2, 0), basis(2, 1)))

(Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[0.]
  [0.]
  [1.]
  [0.]], 1.0)

As predicted the output is the state after swapping which is `|10⟩`